# READ DATA AND CLEANING, ORDERING


In [ ]:
import pandas as pd
import numpy as np
pd.set_option("display.max_rows", None)

In [ ]:
df = pd.read_csv('spot.csv')
df = df.dropna(subset=['event_name'])
df.sort_values(by=["block_timestamp","transaction_index","log_index"], inplace=True)
df.reset_index(drop=True, inplace=True)

# USDC

In [ ]:
df["usdc"]=0
usdc=0

for i in df.index:
    if df["event_name"][i]=="Initialize":
        df["usdc"][i]=0
        
    elif df["event_name"][i]=="Increase liquidity": #i 0 olsa one hell yolu tap
        usdc=usdc+df["token0_in"][i]
        df["usdc"][i]=usdc
             
        
    elif df["event_name"][i]=="Collect":
        usdc=usdc-df["token0_out"][i]
        df["usdc"][i]=usdc
              
    elif df["event_name"][i]=="Swap":
        if str(df["token0_in"][i]) != 'nan':
            usdc=usdc+df["token0_in"][i]
            df["usdc"][i]=usdc
            
                 
        else:
            usdc=usdc-df["token0_out"][i]
            df["usdc"][i]=usdc
         
    elif df["event_name"][i]=="Burn (Remove)":
        df["usdc"][i]=usdc
                 
    else:
        df["usdc"][i]=usdc                             

# SPOT PRICE AND RATE

In [ ]:
df["spot_price"]=0
df["rate"]=0

for i in df.index:
    if df['event_name'][i]=='Initialize':
        df['rate'][i]=0
        df["spot_price"][i]=0
        
    elif df['event_name'][i]=='Increase liquidity':
        if "Swap" not in df.iloc[:i+1]["event_name"].unique() or df["usdc"][i-1]<1: 
            df['rate'][i]=df['token0_in'][i]/df['token1_in'][i]
            df["spot_price"][i]=df['rate'][i]
        else:
            df['rate'][i]=df['rate'][i-1]
            df["spot_price"][i]=df['rate'][i]
            
    
    elif df['event_name'][i]=='Swap':   
        if str(df["token0_in"][i]) != 'nan':
            df['rate'][i]=df['token0_in'][i]*0.997/df['token1_out'][i]
            df["spot_price"][i]=df['rate'][i]
            
        else: 
            df['rate'][i]=(1/(df['token1_in'][i]*0.997/df['token0_out'][i]))
            df["spot_price"][i]=df['rate'][i]
            
    else:
        df['rate'][i]=df['rate'][i-1]
        df["spot_price"][i]=df['rate'][i]